In [ ]:
import requests
import urllib.parse

# Set the AMap API key
key = ""

# Set the origin address
origin_address = ""

# List of addresses (destination addresses)
addresses = [
    "",
    ""
]

# Function to get coordinates
def get_coordinates(address):
    # URL encode the address
    address_encoded = urllib.parse.quote(address)
    # Request geocoding API to get latitude and longitude
    url = f"https://restapi.amap.com/v3/geocode/geo?address={address_encoded}&key={key}"
    response = requests.get(url)
    data = response.json()
    
    # Print detailed API response data
    print(f"Geocoding API response: {data}")
    
    if data['status'] == '1' and data['geocodes']:
        # Get the longitude and latitude of the first matching result
        lon, lat = data['geocodes'][0]['location'].split(',')  # Longitude first, latitude second
        return float(lon), float(lat)  # Return longitude and latitude
    else:
        print(f"Unable to get coordinates for {address}")
        return None

# Function to get walking route
def get_walking_route(origin, addresses):
    origin_coords = get_coordinates(origin)  # Get the coordinates of the origin address
    if not origin_coords:
        return []

    routes = []
    for destination in addresses:
        destination_coords = get_coordinates(destination)  # Get the coordinates of the destination address
        if not destination_coords:
            routes.append((origin, destination, "Unable to calculate walking route: Invalid destination"))
            continue

        # Get coordinates of the destination address
        destination_lon, destination_lat = destination_coords  # Longitude first, latitude second
        destination_str = destination        
        # Build the request URL using the walking route API
        url = f'https://restapi.amap.com/v3/direction/walking?origin={origin_coords[0]},{origin_coords[1]}&destination={destination_lon},{destination_lat}&key={key}'  # **Modified to use longitude and latitude coordinates**
        print(f"Request URL: {url}")
        
        # Make the request
        response = requests.get(url)
        data = response.json()  # The returned data is in JSON format
        
        # Output response data to check for error messages
        # print(f"Walking route API response: {data}")
        
        if data['status'] == '1' and 'route' in data:  # Check if the API response is valid
            # Extract walking route information
            if 'paths' in data['route'] and data['route']['paths']:
                distance = data['route']['paths'][0]['distance']  # Get the walking route distance (unit: meters)
                routes.append((origin, destination_str, distance))
            else:
                routes.append((origin, destination_str, "No walking path found"))
        else:
            routes.append((origin, destination_str, f"Unable to calculate walking route: {data.get('info', 'Unknown error')}"))
    
    return routes

# Get and print the walking route distance for each destination address
routes = get_walking_route(origin_address, addresses)
for origin, destination, distance in routes:
    print(f"The walking route distance from '{origin}' to '{destination}' is {distance} meters")
